In [1]:
from pydrake.all import (DiagramBuilder, 
                         AddMultibodyPlantSceneGraph, 
                         Parser, 
                         LoadModelDirectives,
                         ProcessModelDirectives,
                         AddDefaultVisualization,
                         Meldis, PackageMap, RobotDiagramBuilder
)
from manipulation import ConfigureParser
import os
def plant_builder(use_meshcat = False, use_rohan_scenario = False):
    if use_meshcat:
        meldis = Meldis()
        meshcat = meldis.meshcat
    builder = RobotDiagramBuilder()
    plant = builder.plant()
    scene_graph = builder.scene_graph()
    parser = builder.parser()
    ConfigureParser(parser)
    if use_rohan_scenario:
        # directives_file = './models/iiwa14_rohan_cheap.dmd.yaml'
        directives_file = './models/iiwa14_rohan_cheap.dmd.yaml'
    else:
        directives_file = './models/iiwa14_david_cheap.dmd.yaml'
        

    parser.package_map().AddPackageXml(filename=os.path.abspath("./models/package.xml"))
    directives = LoadModelDirectives(directives_file)
    models = ProcessModelDirectives(directives, plant, parser)
    plant.Finalize()
    if use_meshcat:
        visualizer = AddDefaultVisualization(builder.builder(), meshcat)
    diagram = builder.Build()
    diagram_context = diagram.CreateDefaultContext()
    plant_context = plant.GetMyContextFromRoot(diagram_context)
    diagram.ForcedPublish(diagram_context)
    return plant, scene_graph, diagram, diagram_context, plant_context, meshcat if use_meshcat else None


use_rohan_scenario = True
plant, scene_graph, diagram, diagram_context, plant_context, meshcat = plant_builder(True, use_rohan_scenario)
scene_graph_context = scene_graph.GetMyMutableContextFromRoot(
    diagram_context)

INFO:drake:Meshcat listening for connections at http://localhost:7000


In [2]:
from pydrake.geometry.optimization import Iris, IrisOptions, IrisInConfigurationSpace

from arm_visualization import scenario_loader, Simulator
from pydrake.all import ConvexSet, RandomGenerator
def visualize_arm_at_samples_from_set(convex_set: ConvexSet, num_samples = 10, dt = 0.2, use_rohan_scenario=False):
    np.random.seed(0)
    sample = None
    generator = RandomGenerator(1)
    samples = []
    for i in range(0,num_samples):    
        if sample is not None:
            sample = convex_set.UniformSample(generator, sample)
        else:
            sample = convex_set.UniformSample(generator)
        samples.append(sample)

    arm_components = scenario_loader(use_rohan_scenario=use_rohan_scenario)

    simulator = Simulator(arm_components.diagram)
    simulator.set_target_realtime_rate(1.0)

    context = simulator.get_mutable_context()
    plant_context = arm_components.plant.GetMyContextFromRoot(context)

    arm_components.meshcat_visualizer.StartRecording()
    t = 0
    N = 20
    for sample in samples:
        for i in range(N):
            t += dt/N
            arm_components.plant.SetPositions(plant_context, sample)
            arm_components.plant.SetVelocities(plant_context, np.zeros(7))
            simulator.AdvanceTo(t)

    arm_components.meshcat_visualizer.StopRecording()
    arm_components.meshcat_visualizer.PublishRecording()

In [3]:
from pydrake.all import (RigidTransform,
                         RotationMatrix,
                         InverseKinematics,
                         Solve,
                         Role,
                         VPolytope,
                         HPolyhedron
                         )
import numpy as np
from tqdm import tqdm

import multiprocessing as mp
from functools import partial


import numpy as np

import random
import colorsys
from fractions import Fraction
import itertools
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
from pydrake.all import (Hyperellipsoid,
                         MathematicalProgram,
                         Solve, 
                         le,
                         ge,
                         eq,
                         MosekSolver,
                         VPolytope,
                         PiecewisePolynomial,
                         SolverOptions,
                         CommonSolverOption,
                         HPolyhedron)
import pydrake.symbolic as sym

from pydrake.all import Rgba, TriangleSurfaceMesh, SurfaceTriangle
from scipy.spatial import ConvexHull

from pydrake.all import Sphere

import ipywidgets as widgets


In [4]:
from pydrake.all import Sphere
col_col =  Rgba(0.8, 0.0, 0, 0.5)    
col_free =  Rgba(0.0, 0.8, 0.5, 0.5) 

def showres(qvis):
    plant.SetPositions(plant_context, qvis)
    diagram.ForcedPublish(diagram_context)
    query = plant.get_geometry_query_input_port().Eval(plant_context)
    col = query.HasCollisions()
    if col:
        meshcat.SetObject(f"/drake/visualizer/col",
                                   Sphere(0.2),
                                   col_col)
    else:
        meshcat.SetObject(f"/drake/visualizer/col",
                                   Sphere(0.2),
                                   col_free)
    meshcat.SetTransform(f"/drake/visualizer/col",
                                   RigidTransform(RotationMatrix(),
                                                  np.array([0,0,2])))
    return col

In [5]:
from pydrake.all import SceneGraphCollisionChecker, VisibilityGraph
robot_instances = [plant.GetModelInstanceByName("iiwa"), plant.GetModelInstanceByName("wsg")]
checker = SceneGraphCollisionChecker(model = diagram, 
                    robot_model_instances = robot_instances,
                    edge_step_size = 0.125)

def vgraph(points, checker, parallelize):
    ad_mat = VisibilityGraph(checker.Clone(), np.array(points).T, parallelize = parallelize)
    N = ad_mat.shape[0]
    for i in range(N):
        ad_mat[i,i] = False
    #TODO: need to make dense for now to avoid wierd nx bugs for saving the metis file.
    return  ad_mat
vgraph_handle = partial(vgraph, checker = checker, parallelize = True) 

INFO:drake:Allocating contexts to support implicit context parallelism 10


In [6]:
from pydrake.all import MaxCliqueSolverViaGreedy

def compute_double_greedy_clique_partition(adj_mat, min_cliuqe_size, worklimit =100):
    cliques = []
    done = False
    adj_curr = adj_mat.copy()
    #adj_curr = 1- adj_curr
    #np.fill_diagonal(adj_curr, 0)
    for i in range(adj_curr.shape[0]):
        adj_curr[i,i] = False
    ind_curr = np.arange(adj_curr.shape[0])
    solver = MaxCliqueSolverViaGreedy()
    while not done:
        #val, ind_max_clique_local = solve_max_independent_set_integer(adj_curr, worklimit=worklimit) #solve_max_independet_set_KAMIS(adj_curr, maxtime = 5) #
        maximal_clique_bools = solver.SolveMaxClique(adj_curr)
        ind_max_clique_local = np.where(maximal_clique_bools)[0]
        #non_max_ind_local = np.arange(len(adj_curr))
        #non_max_ind_local = np.delete(non_max_ind_local, ind_max_clique_local, None)
        index_max_clique_global = np.array([ind_curr[i] for i in ind_max_clique_local])
        cliques.append(index_max_clique_global.reshape(-1))
        adj_curr = np.delete(adj_curr, ind_max_clique_local, 0)
        adj_curr = np.delete(adj_curr, ind_max_clique_local, 1)
        ind_curr = np.delete(ind_curr, ind_max_clique_local)
        if len(adj_curr) == 0 or len(cliques[-1])<min_cliuqe_size:
            done = True
    return cliques

In [7]:
import os 
import pickle

Npts = 1000
seed = 5
offset_size = 0.01
with open(f"./tmp/7DOFBINS_{Npts}_{offset_size}_{seed}.pkl", 'rb') as f:
    d = pickle.load(f)
    q_obj = d['q_obj']
    t_obj = d['t_obj']
    q_tot = d['q_tot']
    t_tot = d['t_tot']        
    ad_mat = d['ad_obj']
    ad_tot = d['ad_tot']
    cliques_obj = d['cliques_obj']
    cliques_tot = d['cliques_tot']

In [8]:
def stretch_array_to_3d(arr, val=0.):
    if arr.shape[0] < 3:
        arr = np.append(arr, val * np.ones((3 - arr.shape[0])))
    return arr
def plot_point(point, meshcat_instance, name,
               color=Rgba(0.06, 0.0, 0, 1), radius=0.01):
    meshcat_instance.SetObject(name,
                               Sphere(radius),
                               color)
    meshcat_instance.SetTransform(name, RigidTransform(
        RotationMatrix(), stretch_array_to_3d(point)))

def plot_points(meshcat, points, name, size = 0.05, color = Rgba(0.06, 0.0, 0, 1)):
    for i, pt in enumerate(points):
        n_i = name+f"/pt{i}"
        plot_point(pt, meshcat, n_i, color = color, radius=size)

In [9]:
for i, pts in enumerate(t_obj):
    plot_points(meshcat, pts, f'task_space_pts{i}/')

In [10]:
# import time
# for q in q_obj[0]:
#     showres(q)
#     time.sleep(0.05)

In [11]:
#construct regions in bins
ellipsoids_obj = []
for q_bin, clique_decomp in zip(q_obj, cliques_obj):
    ellipsoids_bin = []
    for cl in clique_decomp:
        if len(cl)>=8:
            points_clique = q_bin[cl]
            ellipsoid = Hyperellipsoid.MinimumVolumeCircumscribedEllipsoid(points_clique.T)
            ellipsoids_bin.append(ellipsoid)
    ellipsoids_obj.append(ellipsoids_bin)

#run iris on ellipsoids

In [12]:
ellipsoids_obj

[[<pydrake.geometry.optimization.Hyperellipsoid at 0x29dabae70>,

In [13]:
# iris_options = IrisOptions()
# iris_options.starting_ellipse = ellipsoids_obj[0][0]
# iris_options.iteration_limit = 1
# hpoly = IrisInConfigurationSpace(plant, plant_context, iris_options)


In [14]:
# visualize_arm_at_samples_from_set(hpoly, 20, use_rohan_scenario=True)

In [16]:
from pydrake.geometry.optimization import Iris, IrisOptions, IrisInConfigurationSpace

i = 0
all_hpolys = []
for a_bin in ellipsoids_obj:
    bin_hpolys = []
    for ellipsoid in tqdm(a_bin):
        i+=1
        iris_options = IrisOptions()
        iris_options.starting_ellipse = ellipsoid
        iris_options.iteration_limit = 1
        try:
            hpoly = IrisInConfigurationSpace(plant, plant_context, iris_options)
        except:
            print("IRIS failed at ", i)
        bin_hpolys.append(hpoly)
    all_hpolys.append(bin_hpolys)

  0%|          | 0/15 [00:00<?, ?it/s]INFO:drake:IrisInConfigurationSpace iteration 0
INFO:drake:IrisInConfigurationSpace: Terminating because the iteration limit 1 has been reached.
  7%|▋         | 1/15 [00:23<05:35, 23.98s/it]INFO:drake:IrisInConfigurationSpace iteration 0
INFO:drake: Checking wsg::collision against bin1::left has already required 50 counter-example searches; still searching...
INFO:drake: Finished checking wsg::collision against bin1::left after 68 counter-example searches.
INFO:drake:IrisInConfigurationSpace: Terminating because the iteration limit 1 has been reached.
 13%|█▎        | 2/15 [00:43<04:39, 21.46s/it]INFO:drake:IrisInConfigurationSpace iteration 0
INFO:drake: Checking wsg::collision against bin1::left has already required 50 counter-example searches; still searching...
INFO:drake: Finished checking wsg::collision against bin1::left after 73 counter-example searches.
INFO:drake:IrisInConfigurationSpace: Terminating because the iteration limit 1 has bee

IRIS failed at  6


INFO:drake:IrisInConfigurationSpace: Terminating because the iteration limit 1 has been reached.
 47%|████▋     | 7/15 [01:50<01:44, 13.03s/it]INFO:drake:IrisInConfigurationSpace iteration 0
INFO:drake: Checking wsg::collision against bin1::left has already required 50 counter-example searches; still searching...
INFO:drake: Finished checking wsg::collision against bin1::left after 54 counter-example searches.
INFO:drake:IrisInConfigurationSpace: Terminating because the iteration limit 1 has been reached.
 53%|█████▎    | 8/15 [02:03<01:31, 13.13s/it]INFO:drake:IrisInConfigurationSpace iteration 0
INFO:drake: Checking wsg::collision against bin0::right has already required 50 counter-example searches; still searching...
INFO:drake: Finished checking wsg::collision against bin0::right after 60 counter-example searches.
INFO:drake:IrisInConfigurationSpace: Terminating because the iteration limit 1 has been reached.
 60%|██████    | 9/15 [02:19<01:24, 14.02s/it]INFO:drake:IrisInConfigurat

In [18]:
bin_index = 0
for bin_hpolys in all_hpolys:
    poly_index = 0
    for hpoly in bin_hpolys:
        print(bin_index, poly_index)
        visualize_arm_at_samples_from_set(hpoly, 20, dt = 0.1, use_rohan_scenario=True)
        poly_index += 1
    bin_index += 1

# 0 3
# 0 4
# 0 5
# 0 6
# 0 8
# 0 9
# 0 12
# 0 13
# 0 14

# 1 5
# 1 7
# 1 10
# 1 11
# 1 12
# 1 13
# 1 14

INFO:drake:Meshcat listening for connections at http://localhost:7015


0 0


INFO:drake:Meshcat listening for connections at http://localhost:7016


0 1


INFO:drake:Meshcat listening for connections at http://localhost:7017


...

1 12


INFO:drake:Meshcat listening for connections at http://localhost:7043


1 13


INFO:drake:Meshcat listening for connections at http://localhost:7044


1 14


In [19]:
# bin_index = 0
# for bin_hpolys in all_hpolys:
poly_index = 0
for hpoly in all_hpolys[1]:
    print(bin_index, poly_index)
    visualize_arm_at_samples_from_set(hpoly, 30, dt = 0.1, use_rohan_scenario=True)
    poly_index += 1

# 1 2 right, not too good
# 1 3 center, top, not too good
# 1 5 right, good
# 1 6 not too good
# 1 7 very good
# 1 8 not too good, at an angle
# 1 10 fantastic, center right
# 1 11 fantasitc, center left
# 1 12 pretty good, on right
# 1 
# 1 14 ok at an angle

INFO:drake:Meshcat listening for connections at http://localhost:7045


2 0


INFO:drake:Meshcat listening for connections at http://localhost:7046


2 1


INFO:drake:Meshcat listening for connections at http://localhost:7047


...

2 12


INFO:drake:Meshcat listening for connections at http://localhost:7058


2 13


INFO:drake:Meshcat listening for connections at http://localhost:7059


2 14


In [20]:
visualize_arm_at_samples_from_set(all_hpolys[1][10], 30, dt = 0.1, use_rohan_scenario=True)
visualize_arm_at_samples_from_set(all_hpolys[1][11], 30, dt = 0.1, use_rohan_scenario=True)

INFO:drake:Meshcat listening for connections at http://localhost:7060
INFO:drake:Meshcat listening for connections at http://localhost:7061


In [21]:
poly_index = 0
for hpoly in all_hpolys[0]:
    print(bin_index, poly_index)
    visualize_arm_at_samples_from_set(hpoly, 30, dt = 0.1, use_rohan_scenario=True)
    poly_index += 1

# 0 11 a little too tall
# 0 12 too much at an angle
# 0 3 pretty good all over the place

# 0 4 very good
# 0 5 very good
# 0 6 center
# 0 7 center all over the place
# 0 9 very good
# 0 13 nice center
# 0 14 nice

INFO:drake:Meshcat listening for connections at http://localhost:7062


2 0


INFO:drake:Meshcat listening for connections at http://localhost:7063


2 1


INFO:drake:Meshcat listening for connections at http://localhost:7064


...

2 12


INFO:drake:Meshcat listening for connections at http://localhost:7075


2 13


INFO:drake:Meshcat listening for connections at http://localhost:7076


2 14


In [27]:
poly_index = 0
for index in [4,5,6,9]: # 13 14
    hpoly = all_hpolys[0][index]
    print(bin_index, index)
    visualize_arm_at_samples_from_set(hpoly, 30, dt = 0.1, use_rohan_scenario=True)

# 1 5 6 9

INFO:drake:Meshcat listening for connections at http://localhost:7105


2 4


INFO:drake:Meshcat listening for connections at http://localhost:7106


2 5


INFO:drake:Meshcat listening for connections at http://localhost:7107


2 6


INFO:drake:Meshcat listening for connections at http://localhost:7108


2 9


In [28]:
from pydrake.geometry.optimization import SaveIrisRegionsYamlFile

In [31]:
bin_dict = {}
bin_dict["RightBin0"] = all_hpolys[1][10]
bin_dict["RightBin1"] = all_hpolys[1][11]

bin_dict["LeftBin0"] = all_hpolys[0][5]
bin_dict["LeftBin1"] = all_hpolys[0][6]
bin_dict["LeftBin2"] = all_hpolys[0][9]
SaveIrisRegionsYamlFile("new_regions.yaml", bin_dict)

In [33]:
for i, convex_set in enumerate(all_hpolys[0]):
    if convex_set.PointInSet(np.array([0, 0.3, 0, -1.8, 0, 1, 1.57])):
        print(i)
        visualize_arm_at_samples_from_set(convex_set, 30, dt = 0.1, use_rohan_scenario=True)